In [15]:
#import the neccesary libraries
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import random
from PIL import Image
import numpy as np
import skimage
import skimage.transform
import os
import tensorflow.keras.backend as K
from skimage.metrics import structural_similarity as ssim
import math

os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
def CreateTestSet(test_set_indices,input_dir_test,output_dir_test):
    """
    Function that reads in the test set images as arrays and scales them between 0 and 1
    parameters:test_set_indices: The indices of the the images in the test set
                 input_dir_test: The directory where the input png images are stored
                output_dir_test: The directory where the output png images are stored
    returns: Test_set_X: an array of input images
             Test_set_y: an array of output images
    """
    Test_set_X = np.zeros((len(test_set_indices), 801, 401, 1), dtype=np.float32)
    Test_set_y = np.zeros((len(test_set_indices), 801, 401, 1), dtype=np.float32)
    for i,n in enumerate(test_set_indices):
        img = Image.open(input_dir_test+'/input_%03d.png' % (n+1))
        in_img_input = tf.keras.preprocessing.image.img_to_array(img)
        in_img_input = skimage.transform.resize(in_img_input , (801 , 401 , 1) , mode = 'constant' , preserve_range = True)
        Test_set_X[i] =in_img_input / 255.0

        img = Image.open(output_dir_test+'/target_%03d.png' % (n+1))
        in_img_output = tf.keras.preprocessing.image.img_to_array(img)
        in_img_output = skimage.transform.resize(in_img_output , (801 , 401 , 1) , mode = 'constant' , preserve_range = True)
        Test_set_y[i] =in_img_output / 255.0    
    print('Done')
    return Test_set_X, Test_set_y

In [2]:
#specify which dataset is used
dataset = 'CIRS'
if dataset == 'Speckle':
    test_set_indices = range(535,669)
    input_dir_test = 'bandLimited'
    output_dir_test = 'groundTruth'
if dataset == 'CIRS':
    test_set_indices = range(11)
    input_dir_test = 'CIRSBandLimited'
    output_dir_test = 'CIRSGroundTruth'
if dataset =='carotid':
    test_set_indices = range(70)
    input_dir_test = 'carotidBandLimited'
    output_dir_test = 'carotidGroundTruth'
Test_set_X, Test_set_y = CreateTestSet(test_set_indices,input_dir_test,output_dir_test)



Done


In [2]:
def log10(x):
  numerator = K.log(x)
  denominator = K.log(K.constant(10, dtype=numerator.dtype))
  return numerator / denominator
#evaluation metrics 
#MSE(mean squared error)
def MSE(y_predict,y_true):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_predict)
    length = float(len(y_true_f))
    diff = y_true_f-y_pred_f
    sum_diff =K.sum(diff*diff)
    mse = sum_diff/length
    return mse
#RMSE(root mean squared error)
#RMSE = sqrt(MSE)
def RMSE(y_predict,y_true):
    rmse = K.sqrt(MSE(y_predict,y_true))
    return rmse
#PSNR(Peak signal to noise ratio)
#PSNR = 20*log_10(MAX_intensity)-10*log_10(MSE)
def PSNR(y_predict,y_true):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_predict)
    psnr = 20*log10(K.max(y_true_f))-10*log10(MSE(y_predict,y_true))
    return psnr
def gcnr(img1, img2):
    _, bins = np.histogram(np.stack((img1, img2)), bins=256)
    f, _ = np.histogram(img1, bins=bins, density=True)
    g, _ = np.histogram(img2, bins=bins, density=True)
    f /= f.sum()
    g /= g.sum()
    return 1 - np.sum(np.minimum(f, g))
def CalculateAverageMetrics(prediction_images,data_set_true):
    """
    Code to calculate the average metric scores of the model on a data set
    parameters: prediction_images: The predicited images from the FBP images
                data_set_true: The ground truth images of the validation/test set
    """
    rmse= []
    psnr = []
    SSIM = []
    for i in range(len(data_set_true)):
        rmse.append(RMSE(prediction_images[i],data_set_true[i]))
        psnr.append(PSNR(prediction_images[i],data_set_true[i]))
        SSIM.append(ssim(prediction_images[i], data_set_true[i],data_range=data_set_true[i].max() - data_set_true[i].min())) 
    avg_rmse = sum(rmse)/len(data_set_true)
    avg_psnr = sum(psnr)/len(data_set_true)
    avg_SSIM = sum(SSIM)/len(data_set_true)
    std_rmse = np.std(rmse)
    std_psnr = np.std(psnr)
    std_SSIM = np.std(SSIM)
    return avg_rmse,std_rmse,avg_psnr,std_psnr,avg_SSIM,std_SSIM

In [3]:
#The list with filenames of the model output npy files for each dataset
Model_output_NPY_file_list_speckle = ['Model_output_test_model1_ultrasound',
                              'Model_output_test_model2_ultrasound',
                              'Model_output_test_model3_ultrasound',
                              'Model_output_test_model4_ultrasound',
                              'Model_output_test_unetmodel1_ultrasound',
                              'Model_output_test_unetmodel2_ultrasound',
                              'Model_output_test_unetmodel3_ultrasound',
                              'Model_output_test_unetmodel4_ultrasound',
                              'Model_output_test_SRCNNmodel1_ultrasound',
                              'Model_output_test_SRCNNmodel2_ultrasound',
                              'Model_output_test_SRCNNmodel3_ultrasound',
                              'Model_output_test_SRCNNmodel4_ultrasound']

Model_output_NPY_file_list_CIRS = ['Model_output_test_CIRS_model1_ultrasound',
                              'Model_output_test_CIRS_model2_ultrasound',
                              'Model_output_test_CIRS_model3_ultrasound',
                              'Model_output_test_CIRS_model4_ultrasound',
                              'Model_output_test_CIRS_unetmodel1_ultrasound',
                              'Model_output_test_CIRS_unetmodel2_ultrasound',
                              'Model_output_test_CIRS_unetmodel3_ultrasound',
                              'Model_output_test_CIRS_unetmodel4_ultrasound',
                              'Model_output_test_CIRS_SRCNNmodel1_ultrasound',
                              'Model_output_test_CIRS_SRCNNmodel2_ultrasound',
                              'Model_output_test_CIRS_SRCNNmodel3_ultrasound',
                              'Model_output_test_CIRS_SRCNNmodel4_ultrasound']

Model_output_NPY_file_list_carotid = ['Model_output_test_carotid_model1_ultrasound',
                              'Model_output_test_carotid_model2_ultrasound',
                              'Model_output_test_carotid_model3_ultrasound',
                              'Model_output_test_carotid_model4_ultrasound',
                              'Model_output_test_carotid_unetmodel1_ultrasound',
                              'Model_output_test_carotid_unetmodel2_ultrasound',
                              'Model_output_test_carotid_unetmodel3_ultrasound',
                              'Model_output_test_carotid_unetmodel4_ultrasound',
                              'Model_output_test_carotid_SRCNNmodel1_ultrasound',
                              'Model_output_test_carotid_SRCNNmodel2_ultrasound',
                              'Model_output_test_carotid_SRCNNmodel3_ultrasound',
                              'Model_output_test_carotid_SRCNNmodel4_ultrasound']

In [7]:
#save bandlimited and groundtruth image for comparison
def SaveBandlimitedGroundtruth(input_dir,folder,output_dir,img_nr,yloc,xloc,patchsize,full_image=False):
    """
    Function that saves a square image or full-image of the groundtruth and bandlimited images
    parameters:
              input_dir: The directory of the input
              folder: The folder wherein the images will be stored
              output_dir: The directory of the output
              img_nr: The image index of the image to be saved
              yloc: The y location of the patch
              xloc: The x location of the patch
              patchsize: the width and height of the square patch
              full_image: When True, save the full image instead of a square
    """
    img_input = Image.open(input_dir+'/input_%03d.png' % (img_nr+1))
    in_img_input = tf.keras.preprocessing.image.img_to_array(img_input)
    if full_image == True:
        prediction_image = in_img_input
    else:
        prediction_image = in_img_input[yloc:yloc+patchsize,xloc:xloc+patchsize]
    prediction_image  = Image.fromarray(prediction_image.squeeze())
    if prediction_image.mode == "F":
        prediction_image = prediction_image.convert('RGB')
    prediction_image.save(str(folder)+'/BandLimited.png')

    img_output = Image.open(output_dir+'/target_%03d.png' % (img_nr+1))
    in_img_output = tf.keras.preprocessing.image.img_to_array(img_output)
    if full_image == True:
        prediction_image = in_img_output
    else:
        prediction_image = in_img_output[yloc:yloc+patchsize,xloc:xloc+patchsize]
    prediction_image  = Image.fromarray(prediction_image.squeeze())
    if prediction_image.mode == "F":
        prediction_image = prediction_image.convert('RGB')
    prediction_image.save(str(folder)+'/GroundTruth.png')

In [8]:
#Calculate and save the metric values for the RMSE, PSNR and SSIM 
metric_array = np.zeros((12,6))
#Run for all datasets
for dataset in ['speckle','CIRS','carotid']:
    if dataset == 'speckle':
        test_set_indices = range(535,669)
        input_dir_test = 'bandLimited'
        output_dir_test = 'groundTruth'
    if dataset == 'CIRS':
        test_set_indices = range(11)
        input_dir_test = 'CIRSBandLimited'
        output_dir_test = 'CIRSGroundTruth'
    if dataset =='carotid':
        test_set_indices = range(70)
        input_dir_test = 'carotidBandLimited'
        output_dir_test = 'carotidGroundTruth'
    #Create arrays for the input and output test set
    Test_set_X, Test_set_y = CreateTestSet(test_set_indices,input_dir_test,output_dir_test)
    for file_nr in range(len(Model_output_NPY_file_list_carotid)):
        if dataset == 'speckle':
            Model_output_NPY_file = Model_output_NPY_file_list_speckle[file_nr]
        if dataset == 'CIRS':
            Model_output_NPY_file = Model_output_NPY_file_list_CIRS[file_nr]
        if dataset == 'carotid':
            Model_output_NPY_file = Model_output_NPY_file_list_carotid[file_nr]
        if 4<=file_nr<=7:
            U_net = True
        else:
            U_net = False
        if U_net == True:
            data_set_true = np.zeros((len(Test_set_y),800,400),dtype=np.float32)
            Model_output = np.load(Model_output_NPY_file+'.npy')
            for i in range(len(Test_set_y)):
                data_set_true[i] = Test_set_y[i,:800,:400].squeeze()
            metric_array[file_nr,:] = CalculateAverageMetrics(Model_output,data_set_true)
        else:
            data_set_true = np.zeros((len(Test_set_y),801,401),dtype=np.float32)
            Model_output = np.load(Model_output_NPY_file+'.npy')
            for i in range(len(Test_set_y)):
                data_set_true[i] = Test_set_y[i].squeeze()
            metric_array[file_nr,:] = CalculateAverageMetrics(Model_output,data_set_true)
        print('Done')
    df = pd.DataFrame(metric_array)
    df.to_excel(excel_writer = "metric_scores_"+dataset+".xlsx")

Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


In [9]:
#img for speckle == 613-535=78
#img for CIRS == 0
#img for Carotid == 44

def Saveimageforallmodels(Model_output_NPY_file_list,folder,img_nr,yloc,xloc,patchsize,full_image=False):
     """
    Function that saves a square image or full-image of the model generated images
    parameters:
              Model_output_NPY_file_list: A list with npy files for all the models for a certain dataset
              folder: The folder wherein the images will be stored
              img_nr: The image index of the image to be saved
              yloc: The y location of the patch
              xloc: The x location of the patch
              patchsize: the width and height of the square patch
              full_image: When True, save the full image instead of a square
    """
    for i in range(len(Model_output_NPY_file_list)):
        Model_output = np.load(Model_output_NPY_file_list[i]+'.npy')
        if full_image == True:
            prediction_image = Model_output[img_nr]
        else:
            prediction_image = Model_output[img_nr,yloc:yloc+patchsize,xloc:xloc+patchsize]
        prediction_image = prediction_image*255.0
        prediction_image  = Image.fromarray(prediction_image)
        if prediction_image.mode == "F":
            prediction_image = prediction_image.convert('RGB')
        prediction_image.save(str(folder)+'/'+str(Model_output_NPY_file_list[i])+'.png')
#Save a specified image patch for each of the datasets 
patchsize = 256 
full_image=True
#Speckle
folder = 'SpeckleComparisonImages'
xloc = 140
yloc = 200
Saveimageforallmodels(Model_output_NPY_file_list_speckle,folder,78,yloc,xloc,patchsize,full_image)

#save the bandlimited and groundtruth images
input_dir = 'bandLimited'
output_dir = 'groundTruth'
SaveBandlimitedGroundtruth(input_dir,folder,output_dir,535+78,yloc,xloc,patchsize,full_image)

#CIRS
xloc = 50
yloc = 190
folder = 'CIRSComparisonImages'
Saveimageforallmodels(Model_output_NPY_file_list_CIRS,folder,0,yloc,xloc,patchsize,full_image)

#save the bandlimited and groundtruth images
input_dir = 'CIRSBandLimited'
output_dir = 'CIRSGroundTruth'
SaveBandlimitedGroundtruth(input_dir,folder,output_dir,0,yloc,xloc,patchsize,full_image)

#carotid
xloc = 0
yloc = 250
folder = 'CarotidComparisonImages'
Saveimageforallmodels(Model_output_NPY_file_list_carotid,folder,44,yloc,xloc,patchsize,full_image)

#save the bandlimited and groundtruth images
input_dir = 'carotidBandLimited'
output_dir = 'carotidGroundTruth'
SaveBandlimitedGroundtruth(input_dir,folder,output_dir,44,yloc,xloc,patchsize,full_image)

In [20]:
def get_coords_images(data_set_true,test_set_indices):
    """
    Function that save the x and y coordinates for the region of interest and the background region
    parameters: data_set_true: This an array with the target images
                Test_set_indices: The indices in the test set that are used
    returns: x_roi,y_roi,x_bg,y_bg
    """
    x_roi = []
    y_roi = []
    x_bg = []
    y_bg = []
    for img_nr in range(len(test_set_indices)):
        img_true = data_set_true[img_nr]
        plt.imshow(img_true,cmap='gray')
        print('select ROI for the image')
        coord_roi = plt.ginput(1)
        x_roi.append(int(coord_roi[0][0]))
        y_roi.append(int(coord_roi[0][1]))
        print('select background region for the image')
        coord_bg = plt.ginput(1)
        x_bg.append(int(coord_bg[0][0]))
        y_bg.append(int(coord_bg[0][1]))
    return x_roi,y_roi,x_bg,y_bg
        
       

In [21]:
 def gCNR_metric(x_roi,y_roi,x_bg,y_bg, Model_output,test_set_indices):
    """
    The function calculates the gCNR metric for the selected images patches in each image
    returns: avg_GCNR_model: the average GCNR for the model images
             std_GCNR_model: the standard deviation of the GCNR for the model images
    """
    GCNR_model = []
    for i in range(len(test_set_indices)):
        img_model = Model_output[i]
        img_roi = img_model[y_roi[i]:y_roi[i]+64,x_roi[i]:x_roi[i]+64]
        img_bg = img_model[y_bg[i]:y_bg[i]+64,x_bg[i]:x_bg[i]+64]
        GCNR_model.append(gcnr(img_roi,img_bg))
    avg_GCNR_model = np.mean(GCNR_model)
    std_GCNR_model = np.std(GCNR_model)
    return avg_GCNR_model,std_GCNR_model

In [22]:
#Calculate and save the GCNR metric
#Create an empty array for the metric values
metric_array = np.zeros((14,2))
#Run for the carotid and CIRS datasets
for dataset in ['carotid','CIRS']:
    if dataset == 'speckle':
        test_set_indices = range(535,669)
        input_dir_test = 'bandLimited'
        output_dir_test = 'groundTruth'
    if dataset == 'CIRS':
        test_set_indices = range(11)
        input_dir_test = 'CIRSBandLimited'
        output_dir_test = 'CIRSGroundTruth'
    if dataset =='carotid':
        test_set_indices = range(70)
        input_dir_test = 'carotidBandLimited'
        output_dir_test = 'carotidGroundTruth'
    #Create the input and output arrays for the test set
    Test_set_X, Test_set_y = CreateTestSet(test_set_indices,input_dir_test,output_dir_test)
    data_set_true = np.zeros((len(Test_set_y),801,401),dtype=np.float32)
    data_set_BL = np.zeros((len(Test_set_y),801,401),dtype=np.float32)
    #Create an array for the target images
    for i in range(len(Test_set_y)):
        data_set_true[i] = Test_set_y[i].squeeze()
    for i in range(len(Test_set_y)):
    #Create an array for the bandwidth limited images
        data_set_BL[i] = Test_set_X[i].squeeze()
    #Get a list of coordinates for each image in the test set
    x_roi,y_roi,x_bg,y_bg = get_coords_images(data_set_true,test_set_indices)
    #Get the avg and std of the GCNR values for each model and save it in an array
    for file_nr in range(len(Model_output_NPY_file_list_carotid)):
        if dataset == 'speckle':
            Model_output_NPY_file = Model_output_NPY_file_list_speckle[file_nr]
        if dataset == 'CIRS':
            Model_output_NPY_file = Model_output_NPY_file_list_CIRS[file_nr]
        if dataset == 'carotid':
            Model_output_NPY_file = Model_output_NPY_file_list_carotid[file_nr]
        Model_output = np.load(Model_output_NPY_file+'.npy')
        metric_array[file_nr,:] =gCNR_metric(x_roi,y_roi,x_bg,y_bg, Model_output,test_set_indices)
    #save the metric values for the target images and bandwidth limited images respectively
    metric_array[12,:] = gCNR_metric(x_roi,y_roi,x_bg,y_bg, data_set_true,test_set_indices)
    metric_array[13,:] = gCNR_metric(x_roi,y_roi,x_bg,y_bg, data_set_BL,test_set_indices)
    #Put the array in a pandas dataframe
    df = pd.DataFrame(metric_array)
    #save the pandas dataframe in an excel file
    df.to_excel(excel_writer = "GCNR_metric_scores"+dataset+".xlsx")

Done
select ROI for the image
select background region for the image
select ROI for the image
select background region for the image
select ROI for the image
select background region for the image
select ROI for the image
select background region for the image
select ROI for the image
select background region for the image
select ROI for the image
select background region for the image
select ROI for the image
select background region for the image
select ROI for the image
select background region for the image
[83, 278, 291, 149, 116, 141, 210, 119] [339, 333, 339, 377, 375, 371, 449, 395] [38, 202, 198, 281, 285, 290, 315, 270] [187, 193, 213, 280, 320, 242, 308, 307]
